In [1]:
import pyspark
from pyspark.sql import SQLContext, SparkSession
from pyspark.sql.functions import *

In [2]:
spark_session = SparkSession.builder \
    .config('spark.sql.extension', 'org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions') \
    .config('spark.jars.packages', 'org.apache.iceberg:iceberg-spark3-runtime:0.13.1,software.amazon.awssdk:bundle:2.15.40,software.amazon.awssdk:url-connection-client:2.15.40,org.apache.spark:spark-avro_2.12:3.2.1,com.amazonaws:aws-java-sdk-bundle:1.12.200,com.amazonaws:aws-java-sdk-pom:1.12.200,org.apache.hadoop:hadoop-aws:3.2.3,com.amazonaws:aws-java-sdk-core:1.12.200') \
    .config('spark.sql.catalog.spark_catalog', 'org.apache.iceberg.spark.SparkCatalog') \
    .config('spark.sql.catalog.spark_catalog.warehouse', 's3a://pyspark-format-010953350464-ap-southeast-2') \
    .config('spark.sql.catalog.spark_catalog.catalog-impl', 'org.apache.iceberg.aws.glue.GlueCatalog') \
    .config('spark.sql.catalog.spark_catalog.io-impl', 'org.apache.iceberg.aws.s3.S3FileIO') \
    .config('spark.sql.catalog.spark_catalog.lock-impl', 'org.apache.iceberg.aws.glue.DynamoLockManager') \
    .config('spark.sql.catalog.spark_catalog.lock.table', 'icebergGlueLockTable') \
    .config('hive.metastore.client.factory.class', 'com.amazonaws.glue.catalog.metastore.AWSGlueDataCatalogHiveClientFactory') \
    .config('hive.metastore.schema.verification', 'false') \
    .config('hive.metastore.schema.verification.record.version', 'false') \
    .config('spark.delta.logStore.class', 'org.apache.spark.sql.delta.storage.S3SingleDriverLogStore') \
    .config('spark.sql.hive.convertMetastoreParquet', 'false') \
    .config('spark.serializer', 'org.apache.spark.serializer.KryoSerializer') \
    .config('spark.hadoop.fs.s3a.aws.credentials.provider', 'org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider') \
    .config("spark.hadoop.datanucleus.autoCreateSchema", 'true') \
    .config("spark.hadoop.datanucleus.schema.autoCreateTables", 'true') \
    .config("spark.hadoop.datanucleus.fixedDatastore", 'false') \
    .getOrCreate()

:: loading settings :: url = jar:file:/usr/local/lib/python3.7/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
org.apache.iceberg#iceberg-spark3-runtime added as a dependency
software.amazon.awssdk#bundle added as a dependency
software.amazon.awssdk#url-connection-client added as a dependency
org.apache.spark#spark-avro_2.12 added as a dependency
com.amazonaws#aws-java-sdk-bundle added as a dependency
com.amazonaws#aws-java-sdk-pom added as a dependency
org.apache.hadoop#hadoop-aws added as a dependency
com.amazonaws#aws-java-sdk-core added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-0039784d-eda6-40ac-b24f-cbdfa5992526;1.0
	confs: [default]
	found org.apache.iceberg#iceberg-spark3-runtime;0.13.1 in central
	found software.amazon.awssdk#bundle;2.15.40 in central
	found software.amazon.eventstream#eventstream;1.0.1 in central
	found software.amazon.awssdk#url-connection-client;2.15.40 in central
	found software.amazon.awssdk#utils;2.15.40 in central
	found org.r

In [3]:
# Create a DataFrame
inputDF = spark_session.createDataFrame(
    [
         ("Italy", "11", "1111", "table", 5, 1648126827, "2015/05/02", "2015", "05", "02"),
         ("Spain", "22", "2222", "chair", 5, 1648126827, "2015/05/02", "2015", "05", "02")        
    ],
    ["marketplace", "review_id", "customer_id", "product_title", "star_rating", "timestamp", "review_date", "year", "month", "day" ]
)

# Print Schema of inputDF 
inputDF.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- review_id: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- product_title: string (nullable = true)
 |-- star_rating: long (nullable = true)
 |-- timestamp: long (nullable = true)
 |-- review_date: string (nullable = true)
 |-- year: string (nullable = true)
 |-- month: string (nullable = true)
 |-- day: string (nullable = true)



In [4]:
inputDF.select(col("review_date"),to_date(col("review_date"),"yyyy/MM/dd").alias("review_date"))

DataFrame[review_date: string, review_date: date]

In [6]:
spark_session.sql(
  "create table if not exists default.amazon_customer_review_iceberg ( \
      marketplace string, \
      review_id string, \
      customer_id string, \
      product_title string, \
      star_rating int, \
      timestamp long , \
      review_date string, \
      year string, \
      month string, \
      day string \
      ) \
      using iceberg \
      location 's3a://pyspark-format-010953350464-ap-southeast-2/amazon_customer_review_iceberg' \
      partitioned by (year,month,day);"
)

Py4JJavaError: An error occurred while calling o64.sql.
: software.amazon.awssdk.core.exception.SdkClientException: Unable to load region from any of the providers in the chain software.amazon.awssdk.regions.providers.DefaultAwsRegionProviderChain@6bbdc94f: [software.amazon.awssdk.regions.providers.SystemSettingsRegionProvider@2e402a69: Unable to load region from system settings. Region must be specified either via environment variable (AWS_REGION) or  system property (aws.region)., software.amazon.awssdk.regions.providers.AwsProfileRegionProvider@8637f40: No region provided in profile: default, software.amazon.awssdk.regions.providers.InstanceProfileRegionProvider@49b200: Unable to contact EC2 metadata service.]
	at software.amazon.awssdk.core.exception.SdkClientException$BuilderImpl.build(SdkClientException.java:98)
	at software.amazon.awssdk.regions.providers.AwsRegionProviderChain.getRegion(AwsRegionProviderChain.java:70)
	at software.amazon.awssdk.awscore.client.builder.AwsDefault

In [7]:
spark_session.sql("INSERT INTO default.amazon_customer_review_iceberg VALUES ('Italy', '11', '1111', 'table', 5, 1648126827, '2015/05/02', '2015', '05', '02')");

Py4JJavaError: An error occurred while calling o64.sql.
: software.amazon.awssdk.core.exception.SdkClientException: Unable to load region from any of the providers in the chain software.amazon.awssdk.regions.providers.DefaultAwsRegionProviderChain@2e168cfd: [software.amazon.awssdk.regions.providers.SystemSettingsRegionProvider@1501e688: Unable to load region from system settings. Region must be specified either via environment variable (AWS_REGION) or  system property (aws.region)., software.amazon.awssdk.regions.providers.AwsProfileRegionProvider@f097772: No region provided in profile: default, software.amazon.awssdk.regions.providers.InstanceProfileRegionProvider@6c334a9e: Unable to contact EC2 metadata service.]
	at software.amazon.awssdk.core.exception.SdkClientException$BuilderImpl.build(SdkClientException.java:98)
	at software.amazon.awssdk.regions.providers.AwsRegionProviderChain.getRegion(AwsRegionProviderChain.java:70)
	at software.amazon.awssdk.awscore.client.builder.AwsDefau